In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Insert warehouse name here
warehouse_name = "OE"
base = "Lucas_Systems_Capstone_Project"

tables = {
    f"{warehouse_name}_Activity": f"../data/database_backups_csv/{warehouse_name}/{warehouse_name}_Activity.csv",
    f"{warehouse_name}_Locations": f"../data/database_backups_csv/{warehouse_name}/{warehouse_name}_Locations.csv",
    f"{warehouse_name}_Products": f"../data/database_backups_csv/{warehouse_name}/{warehouse_name}_Products.csv",
}

column_names = {
    f"{warehouse_name}_Activity": ["ActivityCode","UserID","WorkCode","AssignmentID","ProductID","Quantity","Timestamp","LocationID"],
    f"{warehouse_name}_Locations": ["LocationID","Aisle","Bay","Level","Slot"],
    f"{warehouse_name}_Products": ["ProductID","ProductCode","UnitOfMeasure","Weight","Cube","Length","Width","Height"],
}
#data/database_backups_csv/OE/OE_Activity.csv
dfs = {}

# Run this if there are no column names
for name, fp in tables.items():
    dfs[name] = pd.read_csv(fp, header=None, names=column_names[name])

# Load distance matrix
path = f"../data/distance_matrices/distance_matrix_{warehouse_name}.csv"
Distance = pd.read_csv(path, index_col=0)

for c in Distance.columns:
    Distance[c] = pd.to_numeric(Distance[c], errors="coerce")


In [5]:
for t in [f"{warehouse_name}_Activity", 
          f"{warehouse_name}_Locations", 
          f"{warehouse_name}_Products"]:

    print("=" * 80)
    print(f"Table: {t}")

    df = dfs[t]
    print(f"Dimensions: ({df.shape[0]} rows, {df.shape[1]} columns)\n")

    display(df.head(3))

    schema_df = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "n_missing": df.isna().sum(),
        "n_unique": df.nunique(dropna=True),
    })

    num_df = df.select_dtypes(include="number")
    schema_df["min"] = num_df.min()
    schema_df["max"] = num_df.max()
    schema_df["mean"] = num_df.mean()

    display(schema_df)
    print("\n")


Table: OE_Activity
Dimensions: (96132 rows, 8 columns)



,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID
0,PickPut,419,20,7954566,4289.0,1.0,2025-11-10 11:37:14.160,826367.0
1,AssignmentOpen,64,10,7954429,NaN,NaN,2025-11-10 11:38:34.043,NaN
2,PickPut,419,20,7954541,6592.0,1.0,2025-11-10 11:39:42.330,14524.0


,dtype,n_missing,n_unique,min,max,mean
ActivityCode,object,0,2,NaN,NaN,NaN
UserID,int64,0,40,64.0,504.0,4.050262e+02
WorkCode,int64,0,3,10.0,30.0,2.642034e+01
AssignmentID,int64,0,42241,7717782.0,8042473.0,7.924902e+06
ProductID,float64,894,7691,1.0,57791.0,2.477778e+04
Quantity,float64,894,144,1.0,1143.0,8.453474e+00
Timestamp,object,0,95804,NaN,NaN,NaN
LocationID,float64,894,7669,1.0,8104198.0,8.932247e+05




Table: OE_Locations
Dimensions: (33519 rows, 5 columns)



,LocationID,Aisle,Bay,Level,Slot
0,1,10,24.0,1.0,1.0
1,2,40,5.0,4.0,4.0
2,3,40,9.0,1.0,2.0


,dtype,n_missing,n_unique,min,max,mean
LocationID,int64,0,33519,1.0,8034868.0,795274.768400
Aisle,object,0,52,NaN,NaN,NaN
Bay,float64,1,86,1.0,99.0,23.107107
Level,float64,1,12,1.0,50.0,3.687153
Slot,float64,1,35,1.0,35.0,3.019243




Table: OE_Products
Dimensions: (57671 rows, 8 columns)



,ProductID,ProductCode,UnitOfMeasure,Weight,Cube,Length,Width,Height
0,1,07062B2324X,CA,35.0000,0.938,NaN,NaN,NaN
1,2,0204800418,BX,2.7338,0.263,NaN,NaN,NaN
2,3,07062B1322Q,EA,0.6500,0.027,NaN,NaN,NaN


,dtype,n_missing,n_unique,min,max,mean
ProductID,int64,0,57671,1.0,57671.000,28836.000000
ProductCode,object,0,41981,NaN,NaN,NaN
UnitOfMeasure,object,0,55,NaN,NaN,NaN
Weight,float64,0,5572,0.0,7584.000,4.834848
Cube,float64,0,4009,0.0,421.296,0.623222
Length,float64,57671,0,NaN,NaN,NaN
Width,float64,57671,0,NaN,NaN,NaN
Height,float64,57671,0,NaN,NaN,NaN


In [6]:
display(Distance.head())

dist_long = (
    Distance.stack(dropna=False)
    .rename("distance")
    .reset_index()
    .rename(columns={"level_0": "FromLoc", "level_1": "ToLoc"})
)

display(dist_long.head())


,08|03|||,08|05|||,08|07|||,08|09|||,10|04|||,10|06|||,10|08|||,10|10|||,10|12|||,10|14|||,...,|Start L3,|Start L4,|Start L5,|Start L6,|Start R2,|Start R3,|Start R4,|Start R5,|Start R6,|Start SB
08|03|||,0,414,389,364,304,287,271,255,240,223,...,1094,1068,897,953,1080,1045,1089,913,969,994
08|05|||,25,0,414,389,329,312,296,280,265,248,...,1119,1093,922,978,1104,1070,1113,938,994,1018
08|07|||,50,25,0,414,354,337,321,305,290,273,...,1144,1118,947,1003,1129,1095,1138,963,1019,1043
08|09|||,75,50,25,0,379,362,346,330,315,298,...,1169,1143,972,1028,1154,1119,1163,988,1044,1068
10|04|||,484,459,434,410,0,19,35,51,67,81,...,1120,1074,958,838,929,1133,1087,971,854,767


/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_23989/3226902667.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  Distance.stack(dropna=False)


,FromLoc,ToLoc,distance
0,08|03|||,08|03|||,0
1,08|03|||,08|05|||,414
2,08|03|||,08|07|||,389
3,08|03|||,08|09|||,364
4,08|03|||,10|04|||,304


In [7]:
activity_key = f"{warehouse_name}_Activity"
locations_key = f"{warehouse_name}_Locations"
products_key = f"{warehouse_name}_Products"

# Activity
dfs[activity_key]["ProductID"] = pd.to_numeric(dfs[activity_key]["ProductID"], errors="coerce").astype("Int64")
dfs[activity_key]["Quantity"]  = pd.to_numeric(dfs[activity_key]["Quantity"], errors="coerce").astype("Int64")
dfs[activity_key]["LocationID"] = pd.to_numeric(dfs[activity_key]["LocationID"], errors="coerce").astype("Int64")
dfs[activity_key]["Timestamp"] = pd.to_datetime(dfs[activity_key]["Timestamp"], errors="coerce")
dfs[activity_key]["UserID"] = dfs[activity_key]["UserID"].astype(str)
dfs[activity_key]["WorkCode"] = dfs[activity_key]["WorkCode"].astype(str)
dfs[activity_key]["AssignmentID"] = dfs[activity_key]["AssignmentID"].astype(str)

dfs[activity_key] = dfs[activity_key].dropna(subset=["Timestamp"]).copy()

# Locations
dfs[locations_key]["LocationID"] = pd.to_numeric(dfs[locations_key]["LocationID"], errors="coerce").astype("Int64")
for col in ["Bay", "Level", "Slot"]:
    dfs[locations_key][col] = pd.to_numeric(dfs[locations_key][col], errors="coerce").astype("Int64")

# Products
dfs[products_key]["ProductID"] = pd.to_numeric(dfs[products_key]["ProductID"], errors="coerce").astype("Int64")
dfs[products_key] = dfs[products_key][["ProductID", "ProductCode", "UnitOfMeasure", "Weight", "Cube"]]

Activity = dfs[activity_key]
Locations = dfs[locations_key]
Products = dfs[products_key]


In [8]:
df_work = Activity.copy()
df_work = df_work.sort_values(["UserID", "Timestamp"]).reset_index(drop=True)

g = df_work.groupby("UserID", sort=False)
df_work["Prev_Timestamp"] = g["Timestamp"].shift(1)
df_work["Prev_LocationID"] = g["LocationID"].shift(1)

df_work["Time_Delta_sec"] = (
    df_work["Timestamp"] - df_work["Prev_Timestamp"]
).dt.total_seconds()

df_work.loc[df_work["Time_Delta_sec"] > 180 * 60, "Time_Delta_sec"] = np.nan

Activity_prepped = df_work

In [9]:
display(Activity_prepped.head())

,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID,Prev_Timestamp,Prev_LocationID,Time_Delta_sec
0,PickPut,143,30,7717848,49658,160,2025-09-08 12:11:50.830,35192,NaT,<NA>,NaN
1,PickPut,143,30,7717860,460,50,2025-09-08 12:12:18.127,422,2025-09-08 12:11:50.830,35192,27.297
2,PickPut,143,30,7717908,460,100,2025-09-08 12:15:46.650,422,2025-09-08 12:12:18.127,422,208.523
3,PickPut,143,30,7717921,44547,13,2025-09-08 12:16:30.470,10743,2025-09-08 12:15:46.650,422,43.820
4,PickPut,143,30,7717920,44547,13,2025-09-08 12:18:00.970,10743,2025-09-08 12:16:30.470,10743,90.500


In [10]:
df_joined = Activity_prepped.merge(Products, on="ProductID", how="left")
df_joined = df_joined.merge(Locations, on="LocationID", how="left")

df_joined = df_joined.merge(
    Locations[["LocationID","Aisle","Bay","Level","Slot"]].rename(columns={
        "LocationID": "Prev_LocationID",
        "Aisle": "Prev_Aisle",
        "Bay": "Prev_Bay",
        "Level": "Prev_Level",
        "Slot": "Prev_Slot",
    }),
    on="Prev_LocationID",
    how="left"
)

display(df_joined.head())


,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID,Prev_Timestamp,Prev_LocationID,...,Weight,Cube,Aisle,Bay,Level,Slot,Prev_Aisle,Prev_Bay,Prev_Level,Prev_Slot
0,PickPut,143,30,7717848,49658,160,2025-09-08 12:11:50.830,35192,NaT,<NA>,...,0.0113,0.005,40,19,2,2,NaN,<NA>,<NA>,<NA>
1,PickPut,143,30,7717860,460,50,2025-09-08 12:12:18.127,422,2025-09-08 12:11:50.830,35192,...,0.0300,0.016,40,18,2,1,40,19,2,2
2,PickPut,143,30,7717908,460,100,2025-09-08 12:15:46.650,422,2025-09-08 12:12:18.127,422,...,0.0300,0.016,40,18,2,1,40,18,2,1
3,PickPut,143,30,7717921,44547,13,2025-09-08 12:16:30.470,10743,2025-09-08 12:15:46.650,422,...,0.5000,0.139,40,18,2,2,40,18,2,1
4,PickPut,143,30,7717920,44547,13,2025-09-08 12:18:00.970,10743,2025-09-08 12:16:30.470,10743,...,0.5000,0.139,40,18,2,2,40,18,2,2


In [11]:
df_detailed = df_joined.copy()

df_detailed["Aisle2"] = pd.to_numeric(df_detailed["Aisle"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
df_detailed["Bay2"] = pd.to_numeric(df_detailed["Bay"], errors="coerce").astype("Int64").astype(str).str.zfill(2)

df_detailed["Prev_Aisle2"] = pd.to_numeric(df_detailed["Prev_Aisle"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
df_detailed["Prev_Bay2"] = pd.to_numeric(df_detailed["Prev_Bay"], errors="coerce").astype("Int64").astype(str).str.zfill(2)

SUFFIX = "|||"
df_detailed["LocKey"] = df_detailed["Aisle2"] + "|" + df_detailed["Bay2"] + SUFFIX
df_detailed["PrevLocKey"] = df_detailed["Prev_Aisle2"] + "|" + df_detailed["Prev_Bay2"] + SUFFIX

df_detailed = df_detailed.merge(
    dist_long,
    left_on=["LocKey", "PrevLocKey"],
    right_on=["FromLoc", "ToLoc"],
    how="left"
).rename(columns={"distance": "Travel_Distance"}).drop(columns=["FromLoc", "ToLoc"])

display(df_detailed.head())


,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID,Prev_Timestamp,Prev_LocationID,...,Prev_Bay,Prev_Level,Prev_Slot,Aisle2,Bay2,Prev_Aisle2,Prev_Bay2,LocKey,PrevLocKey,Travel_Distance
0,PickPut,143,30,7717848,49658,160,2025-09-08 12:11:50.830,35192,NaT,<NA>,...,<NA>,<NA>,<NA>,40,19,<NA>,<NA>,40|19|||,<NA>|<NA>|||,NaN
1,PickPut,143,30,7717860,460,50,2025-09-08 12:12:18.127,422,2025-09-08 12:11:50.830,35192,...,19,2,2,40,18,40,19,40|18|||,40|19|||,21.0
2,PickPut,143,30,7717908,460,100,2025-09-08 12:15:46.650,422,2025-09-08 12:12:18.127,422,...,18,2,1,40,18,40,18,40|18|||,40|18|||,0.0
3,PickPut,143,30,7717921,44547,13,2025-09-08 12:16:30.470,10743,2025-09-08 12:15:46.650,422,...,18,2,1,40,18,40,18,40|18|||,40|18|||,0.0
4,PickPut,143,30,7717920,44547,13,2025-09-08 12:18:00.970,10743,2025-09-08 12:16:30.470,10743,...,18,2,2,40,18,40,18,40|18|||,40|18|||,0.0


In [12]:
# diaplay first 50 rows of the final cleaned dataset with userid = 64
display(df_detailed[df_detailed["UserID"] == "64"].head())

,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID,Prev_Timestamp,Prev_LocationID,...,Prev_Bay,Prev_Level,Prev_Slot,Aisle2,Bay2,Prev_Aisle2,Prev_Bay2,LocKey,PrevLocKey,Travel_Distance
90994,AssignmentOpen,64,10,7954429,<NA>,<NA>,2025-11-10 11:38:34.043,<NA>,NaT,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>|<NA>|||,<NA>|<NA>|||,NaN
90995,PickPut,64,20,7955278,3250,1,2025-11-10 12:23:59.540,34343,2025-11-10 11:38:34.043,<NA>,...,<NA>,<NA>,<NA>,42,12,<NA>,<NA>,42|12|||,<NA>|<NA>|||,NaN
90996,PickPut,64,20,7955281,43178,1,2025-11-10 12:26:23.993,14453,2025-11-10 12:23:59.540,34343,...,12,6,7,34,26,42,12,34|26|||,42|12|||,220.0
90997,PickPut,64,20,7955279,5842,1,2025-11-10 12:28:31.110,23294,2025-11-10 12:26:23.993,14453,...,26,6,1,42,05,34,26,42|05|||,34|26|||,514.0
90998,PickPut,64,20,7955276,5842,1,2025-11-10 12:28:31.313,23294,2025-11-10 12:28:31.110,23294,...,5,2,1,42,05,42,05,42|05|||,42|05|||,0.0


In [13]:
# Identify indices of 'AssignmentOpen'
open_indices = df_detailed[df_detailed["ActivityCode"] == "AssignmentOpen"].index

# Identify indices of the first activity immediately following an 'AssignmentOpen'
first_activity_indices = open_indices + 1

# Combine them into one set of indices to remove
to_drop = open_indices.union(first_activity_indices).intersection(df_detailed.index)

# Create the final cleaned dataset
Detailed_Data = df_detailed.drop(to_drop).reset_index(drop=True)

In [14]:
# diaplay first 50 rows of the final cleaned dataset with userid = 64
display(Detailed_Data[Detailed_Data["UserID"] == "64"].head(25))

,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID,Prev_Timestamp,Prev_LocationID,...,Prev_Bay,Prev_Level,Prev_Slot,Aisle2,Bay2,Prev_Aisle2,Prev_Bay2,LocKey,PrevLocKey,Travel_Distance
89723,PickPut,64,20,7955281,43178,1,2025-11-10 12:26:23.993,14453,2025-11-10 12:23:59.540,34343,...,12,6,7,34,26,42,12,34|26|||,42|12|||,220.0
89724,PickPut,64,20,7955279,5842,1,2025-11-10 12:28:31.110,23294,2025-11-10 12:26:23.993,14453,...,26,6,1,42,05,34,26,42|05|||,34|26|||,514.0
89725,PickPut,64,20,7955276,5842,1,2025-11-10 12:28:31.313,23294,2025-11-10 12:28:31.110,23294,...,5,2,1,42,05,42,05,42|05|||,42|05|||,0.0
89726,PickPut,64,20,7955277,8344,1,2025-11-10 12:36:27.187,818,2025-11-10 12:28:31.313,23294,...,5,2,1,41,06,42,05,41|06|||,42|05|||,77.0
89727,PickPut,64,20,7955282,202,1,2025-11-10 12:36:48.807,29361,2025-11-10 12:36:27.187,818,...,6,3,4,41,06,41,06,41|06|||,41|06|||,0.0
89728,PickPut,64,20,7955285,4775,1,2025-11-10 12:39:27.200,15680,2025-11-10 12:36:48.807,29361,...,6,6,4,40,10,41,06,40|10|||,41|06|||,166.0
89729,PickPut,64,20,7955283,3155,1,2025-11-10 12:40:22.077,456,2025-11-10 12:39:27.200,15680,...,10,6,10,39,08,40,10,39|08|||,40|10|||,80.0
89730,PickPut,64,20,7955286,56555,1,2025-11-10 12:41:28.720,14627,2025-11-10 12:40:22.077,456,...,8,2,2,39,19,39,08,39|19|||,39|08|||,206.0
89731,PickPut,64,20,7955284,52919,1,2025-11-10 12:42:05.253,163329,2025-11-10 12:41:28.720,14627,...,19,3,1,39,18,39,19,39|18|||,39|19|||,20.0
89732,PickPut,64,10,7955365,46359,20,2025-11-10 12:56:45.757,54443,2025-11-10 12:54:34.317,2276077,...,34,2,1,36,36,35,34,36|36|||,35|34|||,209.0


In [15]:
# Display column names and data types of the final cleaned dataset
display(Detailed_Data.dtypes)

ActivityCode               object
UserID                     object
WorkCode                   object
AssignmentID               object
ProductID                   Int64
Quantity                    Int64
Timestamp          datetime64[ns]
LocationID                  Int64
Prev_Timestamp     datetime64[ns]
Prev_LocationID             Int64
Time_Delta_sec            float64
ProductCode                object
UnitOfMeasure              object
Weight                    float64
Cube                      float64
Aisle                      object
Bay                         Int64
Level                       Int64
Slot                        Int64
Prev_Aisle                 object
Prev_Bay                    Int64
Prev_Level                  Int64
Prev_Slot                   Int64
Aisle2                     object
Bay2                       object
Prev_Aisle2                object
Prev_Bay2                  object
LocKey                     object
PrevLocKey                 object
Travel_Distanc

In [16]:
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

Detailed_Data.to_parquet(output_dir / f"{warehouse_name.lower()}_detailed.parquet", index=False)
Activity_prepped.to_parquet(output_dir / f"{warehouse_name.lower()}_activity_prepped.parquet", index=False)
df_joined.to_parquet(output_dir / f"{warehouse_name.lower()}_joined.parquet", index=False)

print(f"Successfully exported all {warehouse_name} files to {output_dir}")


Successfully exported all OE files to ../data/processed


In [24]:
# Print number of rows in Activity vs in Detailed_Data, and percentage reduced
nrow_original = Activity.shape[0]
nrow_cleaned = Detailed_Data.shape[0]
retained_pc = nrow_cleaned / nrow_original * 100
print(nrow_original)
print(nrow_cleaned)
print(f"Percentage retained: {retained_pc}")

96132
94402
Percentage retained: 98.20039112886447


# Extra

In [ ]:
df = Activity_prepped.copy()
df = df.dropna(subset=["ProductID", "Time_Delta_sec"]).copy()

df["Prev_ProductID"] = df.groupby("UserID")["ProductID"].shift(1)
df_pairs = df[df["ProductID"] == df["Prev_ProductID"]].copy()

product_pick_times = (
    df_pairs.groupby("ProductID")
            .agg(
                n_pairs=("Time_Delta_sec", "size"),
                avg_pick_time_sec=("Time_Delta_sec", "mean"),
                median_pick_time_sec=("Time_Delta_sec", "median"),
                std_pick_time_sec=("Time_Delta_sec", "std")
            )
            .reset_index()
            .sort_values("ProductID")
)

display(product_pick_times.head())
product_pick_times.to_csv(f"../data/processed/product_pick_times.csv", index=False)

,ProductID,n_pairs,avg_pick_time_sec,median_pick_time_sec,std_pick_time_sec
0,1,294,3.716493,0.2835,34.117869
1,3,95,15.635168,9.0570,16.567655
2,4,37,17.089459,17.6200,12.933894
3,7,5,15.023200,2.3260,26.563315
4,8,2,21.388500,21.3885,29.337153
